# Predict flow - as notebook
as close as possible to predict.py in python folder

In [1]:
import os
import dicom
import scipy
from scipy import misc
import shutil
import numpy as np
from PIL import Image, ImageFont, ImageDraw
import cPickle as pickle

In [2]:
import sys
sys.path.insert(0, '../python')
import ild_helpers as H
import cnn_model as CNN4

Using Theano backend.


In [3]:
#font file imported in top directory
font = ImageFont.truetype( '../fonts/arial.ttf', 20)

In [4]:
namedirtop = 'predict'                         # global directory for predict file
predictout='predicted_results'                 # directory for storing image out after prediction
patchpath='patch_bmp'                          # directory for patches from scan images
jpegpath = 'patch_jpeg'                        # subdirectory name to put images
lungmask='lung_mask'                           # directory with lung mask dicom
lungmaskbmp='bmp'                              # directory to put  lung mask bmp
scanbmp='scan_bmp'                             # directory with bmp from dicom
thrproba = 0.8                                 # threshold for probability prediction
predicted_classes = 'predicted_classes.pkl'    # pickle with predicted classes
predicted_proba= 'predicted_probabilities.pkl' # pickle with predicted probabilities
Xprepkl='X_predict.pkl'                        # pickle with Xfile
Xrefpkl='X_file_reference.pkl'
picklefile='pickle'                            # subdirectory name to colect pkl files resulting from prediction

In [5]:
cwd=os.getcwd()
(cwdtop,tail)=os.path.split(cwd)
print cwdtop
print tail

/Users/peterhirt/datascience/ild-cnn
notebooks


In [6]:
# retarget python directory
tail = 'python'
print cwdtop
print tail

/Users/peterhirt/datascience/ild-cnn
python


In [7]:
path_patient = os.path.join(cwdtop,namedirtop)
print path_patient

/Users/peterhirt/datascience/ild-cnn/predict


In [8]:
patient_list= os.walk(path_patient).next()[1]
print patient_list

['107', '121', '138']


In [30]:
# image  patch format
#to enhance contrast on patch put True
contrast=False

typei='bmp'         # can be either 'bmp' or 'jpg'

# dicom file size in pixels
dimtabx = 512
dimtaby = 512

# patch size in pixels 32 * 32
dimpavx = 32
dimpavy = 32

In [10]:
mini=dimtabx-dimpavx
minj=dimtaby-dimpavy
print mini
print minj

480
480


In [11]:
pxy=float(dimpavx*dimpavy)
print pxy

1024.0


In [12]:
# threshold for patch acceptance
thrpatch = 0.8

In [13]:
# color of labels
red=(255,0,0)
green=(0,255,0)
blue=(0,0,255)
yellow=(255,255,0)
cyan=(0,255,255)
purple=(255,0,255)
white=(255,255,255)

In [14]:
classif ={
'consolidation': 0,
'fibrosis': 1,
'ground_glass': 2,
'healthy': 3,
'micronodules': 4,
'reticulation': 5}

In [15]:
classifc ={
'consolidation': red,
'fibrosis': blue,
'ground_glass': yellow,
'healthy': green,
'micronodules': cyan,
'reticulation': purple}

In [16]:
args  = H.parse_args()                          
train_params = {
     'do' : float(args.do) if args.do else 0.5,        
     'a'  : float(args.a) if args.a else 0.3,          # Conv Layers LeakyReLU alpha param [if alpha set to 0 LeakyReLU is equivalent with ReLU]
     'k'  : int(args.k) if args.k else 4,              # Feature maps k multiplier
     's'  : float(args.s) if args.s else 1,            # Input Image rescale factor
     'pf' : float(args.pf) if args.pf else 1,          # Percentage of the pooling layer: [0,1]
     'pt' : args.pt if args.pt else 'Avg',             # Pooling type: Avg, Max
     'fp' : args.fp if args.fp else 'proportional',    # Feature maps policy: proportional, static
     'cl' : int(args.cl) if args.cl else 5,            # Number of Convolutional Layers
     'opt': args.opt if args.opt else 'Adam',          # Optimizer: SGD, Adagrad, Adam
     'obj': args.obj if args.obj else 'ce',            # Minimization Objective: mse, ce
     'patience' : args.pat if args.pat else 5,         # Patience parameter for early stoping
     'tolerance': args.tol if args.tol else 1.005,     # Tolerance parameter for early stoping [default: 1.005, checks if > 0.5%]
     'res_alias': args.csv if args.csv else 'res'      # csv results filename alias
                }


In [17]:
model = H.load_model()
model.compile(optimizer='Adam', loss=CNN4.get_Obj(train_params['obj']))

In [18]:
# open error file

errorfile = open(path_patient+'/predicterrorfile.txt', 'w') 

## Functions definition

In [21]:
def remove_folder(path):
    """
    to remove folder
    called from genebmp function
    """
    # check if folder exists
    if os.path.exists(path):
        # remove if exists
        shutil.rmtree(path)

In [22]:
def genebmp(dirName):
    """
    Objective: generate patches from dicom files
    Input files:  dicom files 
                  lung_mask directory (if present)  
    Output: scan_bmp directory with .bmp files
            bmp directory with .bmp files insidelung_mask directory 
    
    
    
    """
    print ('generate bmp files from dicom files in :',f)
    
    # directory for patches
    bmp_dir = os.path.join(dirName, scanbmp)
    print bmp_dir
    
    remove_folder(bmp_dir)  
    
    os.mkdir(bmp_dir)
    
    # list dcm files
    fileList = os.listdir(dirName)
    
    for filename in fileList:
        # print(filename)
        if ".dcm" in filename.lower():  # check whether the file's DICOM
            FilesDCM =(os.path.join(dirName,filename))  
            #           
            ds = dicom.read_file(FilesDCM)
            endnumslice = filename.find('.dcm')
            imgcore = filename[0:endnumslice] + '.' + typei
            # imgcore=filename[0:endnumslice]+'.dcm'

            # print imgcore
            bmpfile=os.path.join(bmp_dir,imgcore)
            scipy.misc.imsave(bmpfile, ds.pixel_array)
            # ds.save_as(bmpfile)

        # check if lung mask present
        if lungmask == filename:
            lung_dir = os.path.join(dirName, lungmask)
            lung_bmp_dir = os.path.join(lung_dir,lungmaskbmp)
            lunglist = os.listdir(lung_dir)
            remove_folder(lung_bmp_dir)
            # if lungmaskbmp not in lunglist:
            os.mkdir(lung_bmp_dir)
            # print(lung_bmp_dir)
            for lungfile in lunglist:
                # print(lungfile)
                if ".dcm" in lungfile.lower():  # check whether the file's DICOM
                    lungDCM =os.path.join(lung_dir,lungfile)  
                    dslung = dicom.read_file(lungDCM)
                    endnumslice=lungfile.find('.dcm')
                    lungcore=lungfile[0:endnumslice]+'.'+typei
                    lungcoref=os.path.join(lung_bmp_dir,lungcore)
                    scipy.misc.imsave(lungcoref, dslung.pixel_array)
        


In [28]:
def pavgene (namedirtopcf):
        """ 
        generate patches from scan
        """
        
        print('generate patches on: ',f)
        # print namedirtopcf
        
        namemask1=os.path.join(namedirtopcf,lungmask)
        namemask=os.path.join(namemask1,lungmaskbmp)
        # print namemask
        
        bmpdir = os.path.join(namedirtopcf,scanbmp)
        # print bmpdir
        
        patchpathf=os.path.join(namedirtopcf,patchpath)
        jpegpathf=os.path.join(namedirtopcf,jpegpath)
        
        remove_folder(patchpathf)
        os.mkdir(patchpathf)
        
        remove_folder(jpegpathf)
        os.mkdir(jpegpathf)
        
        listbmp= os.listdir(bmpdir)
        # print(listbmp)
        
        if os.path.exists(namemask):
                listlungbmp= os.listdir(namemask)            
        else:
            tflung = False
            listlungbmp=[]
            
        for img in listbmp:
            # print img
            endnumslice = img.find('.bmp')
            posend = endnumslice
            while img.find('-',posend)==-1:
                posend-=1
            debnumslice = posend+1
            slicenumber = (img[debnumslice:endnumslice])
            # print('sln:',slicenumber,'img:', img,debnumslice,endnumslice 
            
            slns = '_' + str(int(slicenumber)) + '.' + typei
            # print(slns)
            
            for llung in listlungbmp:
                tflung=False
                # print(llung)
                # print(listlungbmp)

                if llung.find(slns) > 0:
                    tflung = True
                    lungfile = os.path.join(namemask,llung)
                    # print(lungfile)
                    imlung = Image.open(lungfile)
                    tablung = np.array(imlung)

                    break
            if not tflung:
                    errorfile.write('lung mask not found ' + slns + ' in: ' + f) 
                    print('lung mask not found ', slns, ' in: ', f)
                    tablung = np.ones((dimtabx, dimtaby), dtype='i')
                     
            bmpfile = os.path.join(bmpdir,img)
            im = Image.open(bmpfile)
            imc = im.convert('RGB')
            tabim = np.array(imc)         
            # pavgene (im,tabim,tablung,slicenumber)
            tabf = np.copy(tabim)
        
            # 
            i=0
            while i <= mini:
                j=0
                # j=maxj
                while j <= minj:
                    # print(i,j)
                    area = 0.0
                    x = 0
                    while x < dimpavx:
                        y = 0
                        while y < dimpavy:
                            if tablung[y+j][x+i] > 0:
                                area = area+1
                            y+=1
                        x+=1
           
                    # check if area above threshold
                    if area / pxy > thrpatch:
             
                        crorig = im.crop((i, j, i + dimpavx, j + dimpavy))
                        imagemax=crorig.getbbox()
                        # detect black patch
                        # print (imagemax)
                        
                        if imagemax!=None:
                            namepatch=patchpathf+'/p_'+slicenumber+'_'+str(i)+'_'+str(j)+'.'+typei
                            if contrast:
                                    tabcont=normi(crorig)
                                    scipy.misc.imsave(namepatch, tabcont)
                            else:
                                crorig.save(namepatch)
                                   #we draw the rectange
                            x=0
                            while x < dimpavx:
                                y=0
                                while y < dimpavy:
                                    tabf[y+j][x+i]=[255,0,0]
                                    if x == 0 or x == dimpavx-1 :
                                        y+=1
                                    else:
                                        y+=dimpavy-1
                                x+=1
                    j+=dimpavy
                i+=dimpavx
            # im = plt.matshow(tabf)
            # plt.colorbar(im,label='with pavage')
            scipy.misc.imsave(jpegpathf+'/'+'s_'+slicenumber+'.bmp', tabf)

## start of processing

In [31]:
# scan thru patient list 

for f in patient_list:

    print 'work on patient ID :',f
    namedirtopcf = os.path.join(path_patient,f)
    print namedirtopcf
    
    if os.path.isdir(namedirtopcf):
        genebmp(namedirtopcf)
        pavgene(namedirtopcf)
        #dataprocessing(namedirtopcf)
        #ILDCNNpredict(namedirtopcf)
        #visua(namedirtopcf)
        print 'completed on: ',f
        print ''

work on patient ID : 107
/Users/peterhirt/datascience/ild-cnn/predict/107
('generate bmp files from dicom files in :', '107')
/Users/peterhirt/datascience/ild-cnn/predict/107/scan_bmp
('generate patches on: ', '107')
completed on:  107

work on patient ID : 121
/Users/peterhirt/datascience/ild-cnn/predict/121
('generate bmp files from dicom files in :', '121')
/Users/peterhirt/datascience/ild-cnn/predict/121/scan_bmp
('generate patches on: ', '121')
completed on:  121

work on patient ID : 138
/Users/peterhirt/datascience/ild-cnn/predict/138
('generate bmp files from dicom files in :', '138')
/Users/peterhirt/datascience/ild-cnn/predict/138/scan_bmp
('generate patches on: ', '138')
completed on:  138



In [ ]:
def fidclass(numero):
    """return class from number"""
    found=False
    for cle, valeur in classif.items():
        
        if valeur == numero:
            found=True
            return cle
      
    if not found:
        return 'unknown'

In [ ]:
def interv(borne_inf, borne_sup):
    """Générateur parcourant la série des entiers entre borne_inf et borne_sup.
    inclus
    Note: borne_inf doit être inférieure à borne_sup"""
    
    while borne_inf <= borne_sup:
        yield borne_inf
        borne_inf += 1

In [ ]:
def tagview(fig,text,pro,x,y):
    """write text in image according to label and color"""
    imgn=Image.open(fig)
    draw = ImageDraw.Draw(imgn)
    col=classifc[text]

    deltay=25*(classif[text]%3)
    deltax=175*(classif[text]//3)
    #print text, col
    draw.text((x+deltax, y+deltay),text+' '+pro,col,font=font)
    imgn.save(fig)

In [ ]:
def tagviews(fig,text,x,y):
    """write simple text in image """
    imgn=Image.open(fig)
    draw = ImageDraw.Draw(imgn)
    draw.text((x, y),text,white,font=font)
    imgn.save(fig)

In [ ]:
def maxproba(proba):
    """looks for max probability in result"""
    lenp = len(proba)
    m=0
    for i in interv(0,lenp-1):
        if proba[i]>m:
            m=proba[i]
    return m

In [ ]:
def remove_folder(path):
    # check if folder exists
    if os.path.exists(path):
         # remove if exists
         shutil.rmtree(path)
         print('this direc exist:',path)

In [ ]:
def loadpkl(f,do):
    # create image directory and load pkl files
    dop =os.path.join(do,predoutdirname)
    # pickle with predicted classes
    preclasspick= os.path.join(dop,precclaf)
    # pickle with predicted probabilities
    preprobpick= os.path.join(dop,precprobf)
    # generate input tables from pickles
    dd = open(preclasspick,'rb')
    my_depickler = pickle.Unpickler(dd)
    preclass = my_depickler.load()
#    preclass[0]=0
#    preclass[1]=1
#    preclass[2]=2
#    preclass[3]=3
#    preclass[4]=4
#    preclass[5]=5
    dd.close()
    dd = open(preprobpick,'rb')
    my_depickler = pickle.Unpickler(dd)
    preprob = my_depickler.load()
    dd.close()  
    return (preclass,preprob)

In [ ]:
listpatient=os.listdir(dirpatientdb)    
listpatient.pop(0)
listpatient121 = []
listpatient121.append(listpatient[1])
print listpatient121

# actual code

In [ ]:
print len(listpatient121)
print listpatient121

In [ ]:
for f in listpatient121:
    print('work on: ',f)
    do = os.path.join(dirpatientoutdb,f)
    print do
    doj = os.path.join(do,imgdirname)
    print doj
    remove_folder(doj)
    os.mkdir(doj)
    (preclass,preprob)=loadpkl(f,do)

In [ ]:
    doj = os.path.join(do,imgdirname)
    remove_folder(doj)
    os.mkdir(doj)
    (preclass,preprob)=loadpkl(f,do)

In [ ]:
from six.moves import cPickle
import cPickle as pickle

In [ ]:
predicted_classes = pickle.load( open( "../pickle/predicted_classes.pkl", "rb" ) )
predicted_probabilities = pickle.load( open( "../pickle/predicted_probabilities.pkl", "rb" ) )
X_file_reference = pickle.load( open( "../pickle/X_file_reference.pkl", "rb" ) )

In [ ]:
predicted_classes

In [ ]:
predicted_probabilities

In [ ]:
X_file_reference[0:5]

In [ ]:
print predicted_classes.shape
print predicted_probabilities.shape
print X_file_reference.shape

In [ ]:
results = np.concatenate((predicted_classes, X_file_reference))

In [ ]:
results[0:5]

In [ ]:
a = [[1],[2],[3]]

In [ ]:
b = [[4],[5],[6]]

In [ ]:
np.concatenate((a,b), axis=1)

In [ ]:
aa = [1,2,3]
bb = [4,5,6]

In [ ]:
np.concatenate((aa.T,bb.T), axis=1)

In [ ]:
aaa = predicted_classes[0:3]
bbb = X_file_reference[0:3]
X = [200,210,220]
Y = [100,150,200]

In [ ]:
print aaa[0]

In [ ]:
c = np.zeros((3, 3))
for i in range(3):
    c[i][0] = aaa[i]
    c[i][1] = X[i]
    c[i][2] = Y[i]

In [ ]:
c

In [ ]:
bbb[0].count('_')

In [ ]:
bbb[0].split('_')

In [ ]:
bbbb = bbb[0].split('.')


In [ ]:
print bbbb[0]

In [ ]:
print bbbb[0].split('_')